## StreamStats API Scraper Automatic

__Description__: Tool to automatically run the [USGS StreamStats tool](https://www.usgs.gov/mission-areas/water-resources/science/streamstats-streamflow-statistics-and-spatial-analysis-tools?qt-science_center_objects=0#qt-science_center_objects) for multiple points within a catchment and return the flow frequency curves and subcatchment boundaries.

__Input__: A shapefile containing the latitude and longitude of points on the stream grid for the specified state (confluence and main stem locations).

__Output__: GeoJSON file containing the delinated catchment boundary and flow frequency data for each point, as well as a CSV file containing the flow frequency data.

*Authors*: sputnam@Dewberry.com & slawler@Dewberry.com

### Load libraries and Python options:

In [1]:
import os
import sys
sys.path.append('../USGStools')
from StreamStats_API_Scraper import*
import geopandas as gpd
from geojson import dump

### Specify the state abbreviation and location of the shapefile: 

In [2]:
state='WI' #The state abbreviation in uppercase

if state=='WI': dont_get_flows=True 

path=r'C:\Users\sputnam\Documents\GitHub\usgs-tools\StreamStats\results\Kewaunee_WI' #Specify the location of the shapefile containing the lat/lon of points on the stream grid
name='Confluences1.shp' #The name of the shapefile

##### Load the shapefile:

In [3]:
use_epsg='4326' #Specify a consistent coordinate reference system

gdf=gpd.read_file(os.path.join(path, name)) #Read the shapefile as a geopandas dataframe
gdf=gdf.set_index('num').copy(deep=True) #Set the index to the confluence number

gdf=gdf.to_crs({'init': 'epsg:{0}'.format(use_epsg)}) #Transform the coordinate reference system of the geodataframe

geom=gdf.geometry #Extract the shapley geometry for the outlets in the shapefile

print(geom.head(2))

num
0    POINT (-87.49781885475109 44.45880879748906)
1    POINT (-87.50268392448642 44.46236384102472)
Name: geometry, dtype: object


### Run the API tool for each point:

In [5]:
pp_dic = {} #Dictionary to store the outlet flow frequency data dictionaries

watershed_poly_dic= {} #Dictionary to store the catchment polygons (catchment boundaries)

pp_fail=[] #List to store outlet locations whose flow frequency/catchment polygons were not calculated

pp_dic, watershed_poly_dic=snappoint_analysis(geom, state, status=True) #Run the snappoint function for all catchment outlets within the shapefile and for the specified state. Option: set status=False to hide print statements

-87.49781885475109 44.458808797489056
Fetched Peak Flows
-87.50268392448642 44.46236384102472
Fetched Peak Flows
-87.51347766080534 44.470081928571524
Fetched Peak Flows
-87.51936098740043 44.46856460166849
Fetched Peak Flows
-87.51147146865151 44.476983184922304
Fetched Peak Flows
-87.51566440648695 44.48590208019736
Fetched Peak Flows
-87.52290847978104 44.479604779744434
Line 28: Expecting value: line 1 column 1 (char 0
while loop: watershed_data count: 1
Fetched Peak Flows
-87.52356056216671 44.480457386571636
Fetched Peak Flows
-87.53085628015832 44.48281440137933
Fetched Peak Flows
-87.52697700617833 44.47743071394129
Line 28: Expecting value: line 1 column 1 (char 0
while loop: watershed_data count: 1
Fetched Peak Flows
-87.53185913774162 44.47179378143023
Fetched Peak Flows
-87.53750692702357 44.48729282525018
Fetched Peak Flows
-87.53719088889065 44.46861849584873
Fetched Peak Flows
-87.53660598576154 44.46722956348235
Fetched Peak Flows
-87.53434045303574 44.46411163741862
Fe

### Construct a summary table of the flow frequency data for each outlet:

In [6]:
if not dont_get_flows: ffdata=ff_summary(pp_dic) #Run this function to construct the summary table for all outlet locations

###  Save the results:

In [7]:
if not dont_get_flows: ffdata.to_csv(os.path.join(path,'StreamStats_FlowFrequency.csv')) #Save the results as a csv

##### As a geojson:

In [8]:
for i in pp_dic.keys():
    watershed_poly_dic[i]['features'][0]['ffcurve']=pp_dic[i]
    
with open(os.path.join(path,'StreamStats_Polygons.geojson'), 'w') as f:
   dump(watershed_poly_dic, f)        

# END